# Topic 05: Data Cleaning in Pandas

Jupyter Notebook extensions: https://towardsdatascience.com/jupyter-notebook-extensions-517fa69d2231

1. Map and Apply
2. Groupby
3. Joining DataFrames
4. Missing Values

## `.map()` and `.apply()`

These are two methods used to run functions across multiple rows/columns of your DataFrame, and there are multiple ways to use each.

In [2]:
import numpy as np
import pandas as pd
!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 

  Created wheel for jupyter-nbextensions-configurator: filename=jupyter_nbextensions_configurator-0.4.1-py2.py3-none-any.whl size=465829 sha256=dff70684f551032863c86252201f6e6185b749c1bc3ac2ac6ac0c111d711d977
  Stored in directory: c:\users\melod\appdata\local\pip\cache\wheels\ca\a9\d7\a25604ecdf245a237594b33911a4af1aff9ec3c8e517f55401
  Created wheel for jupyter-latex-envs: filename=jupyter_latex_envs-1.4.6-py2.py3-none-any.whl size=963399 sha256=f7ced166a5a673eda0960bea59a98ddc52fa1220309058cee9d27bd850740e21
  Stored in directory: c:\users\melod\appdata\local\pip\cache\wheels\cd\51\ee\c0fe3c4463ebd357f6b135fcb58667d84079b3e0e8287ec9ea
Successfully built jupyter-nbextensions-configurator jupyter-latex-envs


[I 15:26:44 InstallContribNbextensionsApp] jupyter contrib nbextension install 
[I 15:26:44 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 15:26:44 InstallContribNbextensionsApp] Installing c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore -> addbefore
[I 15:26:44 InstallContribNbextensionsApp] Making directory: C:\ProgramData\jupyter\nbextensions\addbefore\
[I 15:26:44 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore\addbefore.yaml -> C:\ProgramData\jupyter\nbextensions\addbefore\addbefore.yaml
[I 15:26:44 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore\icon.png -> C:\ProgramData\jupyter\nbextensions\addbefore\icon.png
[I 15:26:44 InstallContribNbextensions

[I 15:26:45 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\latex_envs\static\latex_envs.yaml -> C:\ProgramData\jupyter\nbextensions\latex_envs\latex_envs.yaml
[I 15:26:45 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\latex_envs\static\main_toolbar.png -> C:\ProgramData\jupyter\nbextensions\latex_envs\main_toolbar.png
[I 15:26:45 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\latex_envs\static\README.md -> C:\ProgramData\jupyter\nbextensions\latex_envs\README.md
[I 15:26:45 InstallContribNbextensionsApp] Copying: c:\users\melod\anaconda3\envs\learn-env\lib\site-packages\latex_envs\static\thmsInNb4.js -> C:\ProgramData\jupyter\nbextensions\latex_envs\thmsInNb4.js
[I 15:26:45 InstallContribNbextensionsApp] Making directory: C:\ProgramData\jupyter\nbextensions\latex_envs\doc
[I 15:26:45 InstallContribNbextensionsApp] Copying: c:\users\mel

In [3]:
car_df = pd.read_csv('http://faculty.marshall.usc.edu/gareth-james/ISL/Auto.csv', na_values='?')

In [ ]:
car_df.head()

In [ ]:
car_df.describe()

### a) Broadcasting simple operations

i.e. convert `year` into its full year by adding `1900` to each value (we're assuming that the year range of 70-82 is in the 1900s)

In [ ]:
car_df['year'] + 1900 # not destructive, we'd have to reassign the new values

### b) Boolean masking

(also useful to understand this for filtering dataframes)

In [ ]:
car_df[['mpg', 'cylinders', 'displacement']] # % 2 == 0

In [ ]:
car_df['weight'] > 2800 # outputs a series of booleans
# showing whether each value meets the condition set

We can use this to create new, perhaps more useful columns!

In [ ]:
car_df['heavy'] = car_df['weight'] > 2800

In [ ]:
car_df.head()

### c. The `.map()` function

`.map()` only works on Series, or on single columns!

You can supply a condition directly into the lambda, or define specific functions.

In [ ]:
car_df['year'].map(lambda y: y < 80)

In [ ]:
# tangent - for car names - let's see what names exist
# unique(), nunique(), value_counts()
car_df['name']

In [ ]:
car_df['name'].map(lambda n: 'ford' in n)

In [ ]:
# for non-Boolean outputs, you can define a function to feed into the lambda

def car_brand(value):
    if 'ford' in value:
        return 'Ford'
    elif 'buick' in value:
        return 'Buick'
    elif 'chevrolet' in value:
        return 'Chevrolet'
    else:
        return 'Other'
    

In [ ]:
car_df['name'].map(lambda n: car_brand(n))

### d. The `.apply()` function

This works on a DataFrame, or on multiple columns of your DataFrame.


Before that, `applymap()`, **a different method**, is like `.map()` but you can do functions across multiple columns or on an entire DataFrame. 


i.e. turning all your data into strings

In [ ]:
car_df.applymap(lambda value: str(value)) # ['displacement']

# remember this isn't destructive until you reassign it to the variable!

In [ ]:
car_df.columns

`.apply()` allows us to perform specific functions that take different columns as inputs. For example, if we want to engineer a new variable that's some combination of other variables. I usually define the function I want to use in `.apply()` taking in a row as its argument.


Let's create a variable that finds the horsepower per weight of the car.

In [ ]:
def horsepower_per_weight(row):
    return row['horsepower']/row['weight']

In [ ]:
car_df.apply(lambda r: horsepower_per_weight(r), axis=1) # axis = 1 is for columns

## `groupby()` methods for aggregation

To get aggregated information about subsets of your data. Pretty much every time you use a groupby, you have to pair it with an aggregation function.


Some common aggregations:
* `.min()`: returns the minimum value for each column by group  
* `.max()`: returns the maximum value for each column by group  
* `.mean()`: returns the average value for each column by group  
* `.median()`: returns the median value for each column by group  
* `.count()`: returns the count of each column by group

In [ ]:
car_df.head()

In [ ]:
car_df.groupby('cylinders').mean()

## Combining DataFrames

### `pd.concat()` and `pd.join()`

`pd.concat` stacks DataFrames together while maintaining columns or index.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])


df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'], 
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
pd.concat([df1, df2, df3])

In [ ]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])
df4

In [ ]:
df1

In [ ]:
pd.concat([df1, df4], join='inner', axis=1) 

You can supply the `join` argument to `pd.concat`, but Pandas also has a `.join()` method that can be used interchangably in most cases.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join


There are 4 main types of joins: inner, left, right and outer. This is the same kind of join that SQL uses! https://www.w3schools.com/sql/sql_join.asp

- An inner join only keeps the rows that have indexes in both tables.
- Left join keeps all rows in the left table and supplements that information with info from the right table
- Right join does the opposite of a left join
- Outer join returns a combination of all rows

Also, `pd.merge` exists: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge

In [ ]:
pd.concat([df1, df4], join='outer', axis=1) 

In [ ]:
df1.join(df4, how='left', rsuffix='4') # why do we need the rsuffix???

## Missing Data

Definitely read up on the different ways you can go about filling in missing data.

`df.isna()`, `df.fillna()`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [ ]:
test = pd.concat([df1, df4], join='outer', axis=1).drop(['B', 'D'], axis=1)
test

In [ ]:
test.isna().sum()

In [ ]:
test.fillna(0)

In [ ]:
# filling NaNs per column
na_values_per_column = {'A': 3, 'C': 2, 'F': 4}
test.fillna(value=na_values_per_column)

## To-Do:

- Attempt `Project: Data Cleaning - Lab`